# Traffic Application Testing Code

This notebook is designed to communicate with the Traffic Application server and perform a series of tests. You should start the server with a copy of the clean database and execute the core code and each of the tests. If a test fails, it is necessary to restart the server and re-run the test to ensure that the test did not fail because of the state of the server as a result of running previous tests. Each time you restart the server ensure that it begins with the clean version of the database.

In [1]:
# The core code.
import random
import requests
import xml.etree.ElementTree as ET
import base64

def parse_actions(root):
    actions = []
    for child in root.findall('action'):
        type = child.find('type')
        if type.text == 'refill':
            where = child.find('where')
            what = child.find('what')
            actions.append(['refill',[where.text,base64.b64decode(what.text).decode('ascii')]])
        if type.text == 'redirect':
            where = child.find('where')
            actions.append(['redirect',[where.text]])
        if type.text == 'append':
            where = child.find('where')
            what = child.find('what')
            actions.append(['refill',[where.text,base64.b64decode(what.text).decode('ascii')]])
    return actions

def request2server(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]))
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        root=ET.fromstring(r.text)
        act = parse_actions(root)
        return [[ucookie,mcookie],act]
    except:
        return [[ucookie,mcookie],[['status', [-1]]]]
    
def find_redirect(act):
    for a in act:
        if a[0] == 'redirect':
            return a[1][0]
    return None

def find_refill(act,where):
    for a in act:
        if a[0] == 'refill':
            if a[1][0] == where:
                return a[1][1]
    return None

def randparam():
    return "&randn="+str(random.randrange(10000,99999))

def do_login(cookies,user,pasw):
    return request2server("http://localhost:8081/action?command=login&usernameinput="+user+"&passwordinput="+pasw+randparam(),cookies)

def do_add(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:8081/action?command=add&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_undo(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:8081/action?command=undo&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_summary(cookies):
    return request2server("http://localhost:8081/action?command=summary"+randparam(),cookies)

def do_logout(cookies):
    return request2server("http://localhost:8081/action?command=logout"+randparam(),cookies)

In [2]:
def checked_login(test,cookies,user,pasw):
    [cookies,act] = do_login(cookies, user,pasw)
    where = find_redirect(act)
    if where == None:
        print("Test "+str(test)+" Expected redirect.")
        return [cookies,act,False]
    if where != '/page.html':
        print("Test "+str(test)+" Expected /page.html got {"+where+"}")
        return [cookies,act,False]
    return [cookies,act,True]

def checked_logout(test,cookies):
    [cookies,act] = do_logout(cookies)     
    where = find_redirect(act)
    if where != '/index.html':
        print("Test "+str(test)+" Expected /index.html got {"+where+"}")
        return [cookies,act,False] 
    return [cookies,act,True]

In [3]:
# Test 1 - Simple login
def test1():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(1,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return

        [cookies,act,flag] = checked_logout(1,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 1 Passed")
    except:
        print("Test 1 Exception Caused.")


In [4]:
# Test 2 - Bad login
def test2():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1","passwor1")
        where = find_redirect(act2)
    
        if where != None:
            print("Test 2 Unexpected redirect.")
            return
        
        message = find_refill(act2,'message')
    
        if message == None:
            print("Test 2 No message present")
            return
        
        [cookies,act,flag] = checked_logout(2,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 2 Passed")
    except:
        print("Test 2 Exception Caused.")
        

In [5]:
# Test 3 - Injection attack
def test3():
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1",'"'+' OR "0"="0')
        
        where = find_redirect(act)
        if where == None:
            message = find_refill(act,'message')
    
            if (message == None):
                print("Test 3 No failure message present when not re-directing.")
                return
        else:
            print("Test 3 Did not expect to be re-directed")
            return
        
        [cookies,act,flag] = checked_logout(3,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 3 Passed")
    except:
        print("Test 3 Exception Caused.")

In [6]:
# Test 4 - Simple add
def test4():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(4,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        [cookies,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 4 Unexpected redirect.(2)")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 4 No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 4 No total message present")
            return
        
        if total != "1":
            print("Test 4 Total not expected value")
            return
        
        [cookies,act,flag] = checked_logout(4,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 4 Passed")
    except:
        print("Test 4 Exception Caused.")

In [7]:
# Test 5 - Looping Add
def test5():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(5,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if where != None:
            print("Test 5 Unexpected redirect.(2)")
            return
        
          message = find_refill(act,'message')
    
          if message == None:
            print("Test 5 No acknowledge message present")
            return
        
          total = find_refill(act,'total')
    
          if total == None:
            print("Test 5 No total message present")
            return
        
          if total != str(i):
            print("Test 5 Total not expected value {"+str(i)+"}")
            return
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(5,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 5 Passed")
    except:
        print("Test 5 Exception Caused.")

In [8]:
# Test 6 - Per session summary
def test6():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(6,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if where != None:
            print("Test 6 Unexpected redirect.(2)")
            return
        
          message = find_refill(act,'message')
    
          if message == None:
            print("Test 6 No acknowledge message present")
            return
        
          total = find_refill(act,'total')
    
          if total == None:
            print("Test 6 No total message present")
            return
        
          if total != str(i):
            print("Test 6 Total not expected value {"+str(i)+"} Initial Step")
            return
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(6,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        cookies = ['','']
        [cookies,act,flag] = checked_login(6,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if where != None:
            print("Test 6 Unexpected redirect.(2)")
            return
        
          message = find_refill(act,'message')
    
          if message == None:
            print("Test 6 No acknowledge message present")
            return
        
          total = find_refill(act,'total')
    
          if total == None:
            print("Test 6 No total message present")
            return
        
          if total != str(i):
            print("Test 6 Total not expected value {"+str(i)+"} Second Step")
            return
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(6,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return
        
        print("Test 6 Passed")
    except:
        print("Test 6 Exception Caused.")

In [9]:
# Test 7 - Per session summary with mixed traffic
def test7():
    types=["car","bicycle","bus","bus","car","van","motorbike"]
    length = len(types)
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(7,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        i = 1
        
        for type in types:
            
          [cookies_x,act] = do_add(cookies,"road",1,type)
      
          where = find_redirect(act)
    
          if where != None:
            print("Test 7 Unexpected redirect.(2)")
            return
        
          message = find_refill(act,'message')
    
          if message == None:
            print("Test 7 No acknowledge message present")
            return
        
          total = find_refill(act,'total')
    
          if total == None:
            print("Test 7 No total message present")
            return
        
          if total != str(i):
            print("Test 7 Total not expected value {"+str(i)+"} Initial Step")
            return
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(7,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 7 Passed")
    except:
        print("Test 7 Exception Caused.")

In [10]:
# Test 8 - Unknown vehicle type, step 1
def test8():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(7,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        initial_total = find_refill(act,'total')
            
        [cookies_x,act] = do_add(cookies,"road",1,"caravan")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 8 Unexpected redirect.(2)")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 8 No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if total != None:
            if total != initial_total:
                print("Test 8 Total not expected value, should not increase.")
                return
        
        [cookies,act,flag] = checked_logout(8,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 8 Passed")
    except:
        print("Test 8 Exception Caused.")

In [11]:
# Test 9 - Unknown vehicle type, step 2
def test9():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(9,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        initial_total = find_refill(act,'total')
            
        [cookies_x,act] = do_add(cookies,"road",1,"caravan")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 9 Unexpected redirect.(2)")
            return
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 9 Unexpected redirect.(2)")
            return
        
        total = find_refill(act,'total')
    
        if total != None:
            if int(total) != int(initial_total)+1:
                print("Test 9 Total not expected value, should increase by 1 only. Was {"+total+"}")
                return
        
        [cookies,act,flag] = checked_logout(9,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 9 Passed")
    except:
        print("Test 9 Exception Caused.")

In [12]:
# Test 10 - Simple undo
def test10():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(10,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        # entry to get starting point
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 10 Unexpected redirect.(2)")
            return
        
        initial_total = find_refill(act,'total')
        
        # add one
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        # remove one
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 10 Unexpected redirect.(2)")
            return
        
        total = find_refill(act,'total')
        
        if total == None:
            print("Test 10 Total not found.")
            return
    
        if total != initial_total:
            # should have only removed one, not both.
            print("Test 10 Total not expected value, should not increase. Was {"+total+"}")
            return
        
        [cookies,act,flag] = checked_logout(10,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 10 Passed")
    except:
        print("Test 10 Exception Caused.")

In [13]:
# Test 11 - Simple undo more cars than seen
def test11():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(11,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
    
            
        [cookies_x,act] = do_add(cookies,"elsewhere",1,"car")
        
        initial_total = find_refill(act,'total')
        
        [cookies_x,act] = do_undo(cookies,"only time used",1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 11 Unexpected redirect.(2)")
            return
        
        total = find_refill(act,'total')
        
        if total == None:
            print("Test 11 Total not found.")
            return
    
        if total != initial_total:
            print("Test 11 Total not expected value, should not change. Was {"+total+"}")
            return
        
        [cookies,act,flag] = checked_logout(11,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 11 Passed")
    except:
        print("Test 11 Exception Caused.")

In [14]:
# Test 12 - Undo wrong kind of vehicle
def test12():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(12,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
    
            
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 12 Unexpected redirect.(2)")
            return
        
        initial_total = find_refill(act,'total')
        
        [cookies_x,act] = do_undo(cookies,"road",1,"bus")
        
        total = find_refill(act,'total')
        
        if int(total)!=int(initial_total):
            print("Test 12 Total not expected value, should not decrease. Was {"+total+"}")
            return
               

        
        [cookies,act,flag] = checked_logout(12,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 12 Passed")
    except:
        print("Test 12 Exception Caused.")

In [15]:
# Test 13 - Just because I'm a rational scientist does n't mean I'm prepared to pick a fight with fate.
def test13():
    print("Test 13 Passed")


In [16]:
# Test 14 - Add out of range occupancy
def test14():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(14,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        initial_total = find_refill(act,'total')
            
        [cookies_x,act] = do_add(cookies,"road",5,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 14 Unexpected redirect.(2)")
            return

        message = find_refill(act,'message')
        
        if message == None:
            print("Test 14 No error message present")
            return
        
        total = find_refill(act,'total')
        
        if total != None:
            if total != initial_total:
                print("Test 14 Total not expected value, should be change. Was {"+total+"}")
                return
        
        [cookies,act,flag] = checked_logout(14,cookies)        
        where = find_redirect(act)
        
        if flag != True:
            return

        print("Test 14 Passed")
    except:
        print("Test 14 Exception Caused.")

In [17]:
# Test 15 - Undo occupancy out of range
def test15():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(15,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return
    
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)

        
        initial_total = find_refill(act,'total')
        
        
        [cookies_x,act] = do_undo(cookies,"road",5,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 15 Unexpected redirect.(2)")
            return
    
        message = find_refill(act,'message')
        
        if message == None:
            print("Test 15 No error message present")
            return
        
        total = find_refill(act,'total')
        
        if total != None:   
            if total != initial_total:
                print("Test 15 Total not expected value, should not change. Was {"+total+"}")
                return
        
        [cookies,act,flag] = checked_logout(15,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 15 Passed")
    except:
        print("Test 15 Exception Caused.")

In [18]:
# Test 16 - Injection into undo location
def test16():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(16,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
             
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        initial_total = find_refill(act,'total')
        
        
        [cookies_x,act] = do_undo(cookies,'"OR ""=""',1,"car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 16 Unexpected redirect.(2)")
            return
        
        total = find_refill(act,'total')
        
        if total != None:
            if total != initial_total:
                print("Test 16 Total not expected value, should not change. Was {"+total+"}")
                return
        
        [cookies,act,flag] = checked_logout(16,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 16 Passed")
    except:
        print("Test 16 Exception Caused.")

In [19]:
# Test 17 - Injection into undo type
def test17():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(17,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
             
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        initial_total = find_refill(act,'total')
        
        [cookies_x,act] = do_undo(cookies,"road",'1','" OR ""="')
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 17 Unexpected redirect.(2)")
            return
        
        total = find_refill(act,'total')
        
        if total != None:
            if total != initial_total:
                print("Test 17 Total not expected value, should not change. Was {"+total+"}")
                return
            
        message = find_refill(act,'message')
        if message == None:
                print("Test 17 Message not found")
                return
        
        [cookies,act,flag] = checked_logout(16,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 17 Passed")
    except:
        print("Test 17 Exception Caused.")
        

In [20]:
# Test 18 - summary present
def test18():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(18,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
             
        [cookies_x,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if sum_car == None:
            print("Test 18 sum_car not present")
            return
            
        if sum_car != "0":
            print("Test 18 sum_car not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_taxi = find_refill(act,'sum_taxi')
        if sum_taxi == None:
            print("Test 18 sum_taxi not present")
            return
            
        if sum_taxi != "0":
            print("Test 18 sum_taxi not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if sum_bicycle == None:
            print("Test 18 sum_bicycle not present")
            return
            
        if sum_bicycle != "0":
            print("Test 18 sum_bicycle not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_van = find_refill(act,'sum_van')
        if sum_van == None:
            print("Test 18 sum_van not present")
            return
            
        if sum_van != "0":
            print("Test 18 sum_van not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_truck = find_refill(act,'sum_truck')
        if sum_truck == None:
            print("Test 18 sum_truck not present")
            return
            
        if sum_truck != "0":
            print("Test 18 sum_truck not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if sum_motorbike == None:
            print("Test 18 sum_motorbike not present")
            return
            
        if sum_motorbike != "0":
            print("Test 18 sum_motorbike not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_bus = find_refill(act,'sum_bus')
        if sum_car == None:
            print("Test 18 sum_bus not present")
            return
            
        if sum_bus != "0":
            print("Test 18 sum_bus not expected value, should be 0. Was {"+total+"}")
            return
        
        sum_other = find_refill(act,'sum_other')
        if sum_other == None:
            print("Test 18 sum_other not present")
            return
            
        if sum_other != "0":
            print("Test 18 sum_other not expected value, should be 0. Was {"+total+"}")
            return
        
        
        [cookies,act,flag] = checked_logout(18,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 18 Passed")
    except:
        print("Test 18 Exception Caused.")
        

In [21]:
# Test 19 - summary present with some added traffic
def test19():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(19,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        # ignoring any errors here
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"car")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"motorbike")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle") 
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"taxi")
        [cookies,act] = do_add(cookies,"nowhere",3,"taxi")
        [cookies,act] = do_add(cookies,"road road",2,"van")
        [cookies,act] = do_add(cookies,"nowhere",4,"truck")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",3,"other")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if sum_car == None:
            print("Test 19 sum_car not present")
            return
            
        if sum_car != "4":
            print("Test 19 sum_car not expected value, should be 4. Was {"+sum_car+"}")
            return
        
        sum_taxi = find_refill(act,'sum_taxi')
        if sum_taxi == None:
            print("Test 19 sum_taxi not present")
            return
            
        if sum_taxi != "2":
            print("Test 19 sum_taxi not expected value, should be 2. Was {"+sum_taxi+"}")
            return
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if sum_bicycle == None:
            print("Test 19 sum_bicycle not present")
            return
            
        if sum_bicycle != "3":
            print("Test 19 sum_bicycle not expected value, should be 3. Was {"+sum_bicycle+"}")
            return
        
        sum_van = find_refill(act,'sum_van')
        if sum_van == None:
            print("Test 19 sum_van not present")
            return
            
        if sum_van != "1":
            print("Test 19 sum_van not expected value, should be 1. Was {"+sum_van+"}")
            return
        
        sum_truck = find_refill(act,'sum_truck')
        if sum_truck == None:
            print("Test 19 sum_truck not present")
            return
            
        if sum_truck != "1":
            print("Test 19 sum_truck not expected value, should be 1. Was {"+sum_truck+"}")
            return
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if sum_motorbike == None:
            print("Test 19 sum_motorbike not present")
            return
            
        if sum_motorbike != "1":
            print("Test 19 sum_motorbike not expected value, should be 1. Was {"+sum_motorbike+"}")
            return
        
        sum_bus = find_refill(act,'sum_bus')
        if sum_car == None:
            print("Test 19 sum_bus not present")
            return
            
        if sum_bus != "3":
            print("Test 19 sum_bus not expected value, should be 3. Was {"+sum_bus+"}")
            return
        
        sum_other = find_refill(act,'sum_other')
        if sum_other == None:
            print("Test 19 sum_other not present")
            return
            
        if sum_other != "1":
            print("Test 19 sum_other not expected value, should be 1. Was {"+sum_other+"}")
            return
        
        [cookies,act,flag] = checked_logout(19,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 19 Passed")
    except:
        print("Test 19 Failed - Exception Caused.")

In [22]:
# Test 20 - summary present with some added traffic including undo
def test20():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(20,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return
        
        # ignoring any errors here
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"car")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_undo(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"motorbike")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle") 
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"taxi")
        [cookies,act] = do_add(cookies,"nowhere",3,"taxi")
        [cookies,act] = do_undo(cookies,"main",2,"van")
        [cookies,act] = do_add(cookies,"nowhere",4,"truck")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",3,"other")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_add(cookies,"lane",2,"other")
        [cookies,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if sum_car == None:
            print("Test 20 sum_car not present")
            return
            
        if sum_car != "2":
            print("Test 20 sum_car not expected value, should be 2. Was {"+sum_car+"}")
            return
        
        sum_taxi = find_refill(act,'sum_taxi')
        if sum_taxi == None:
            print("Test 20 sum_taxi not present")
            return
            
        if sum_taxi != "2":
            print("Test 20 sum_taxi not expected value, should be 2. Was {"+sum_taxi+"}")
            return
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if sum_bicycle == None:
            print("Test 20 sum_bicycle not present")
            return
            
        if sum_bicycle != "3":
            print("Test 20 sum_bicycle not expected value, should be 3. Was {"+sum_bicycle+"}")
            return
        
        sum_van = find_refill(act,'sum_van')
        if sum_van == None:
            print("Test 20 sum_van not present")
            return
            
        if sum_van != "0":
            print("Test 20 sum_van not expected value, should be 0. Was {"+sum_van+"}")
            return
        
        sum_truck = find_refill(act,'sum_truck')
        if sum_truck == None:
            print("Test 20 sum_truck not present")
            return
            
        if sum_truck != "1":
            print("Test 20 sum_truck not expected value, should be 1. Was {"+sum_truck+"}")
            return
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if sum_motorbike == None:
            print("Test 20 sum_motorbike not present")
            return
            
        if sum_motorbike != "1":
            print("Test 20 sum_motorbike not expected value, should be 1. Was {"+sum_motorbike+"}")
            return
        
        sum_bus = find_refill(act,'sum_bus')
        if sum_car == None:
            print("Test 20 sum_bus not present")
            return
            
        if sum_bus != "3":
            print("Test 20 sum_bus not expected value, should be 3. Was {"+sum_bus+"}")
            return
        
        sum_other = find_refill(act,'sum_other')
        if sum_other == None:
            print("Test 20 sum_other not present")
            return
            
        if sum_other != "2":
            print("Test 20 sum_other not expected value, should be 2. Was {"+sum_other+"}")
            return
        
        [cookies,act,flag] = checked_logout(20,cookies)        
        where = find_redirect(act) 
        if flag != True:
            return

        print("Test 20 Passed")
    except:
        print("Test 20 Exception Caused.")

In [23]:
# Test 21 - two users different cookies
def test21():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(21,cookies1,"test1","password1")
        if flag1!=True:
            return
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(21,cookies2,"test2","password2")
        if flag1!=True:
            return
        
        if cookies1[0] == cookies2[0]:
            print("Test 21 Cookie username matches for dual login.")
            return
        
        if cookies1[1] == cookies2[1]:
            print("Test 21 Cookie magic matches for dual login.")
            return      

        [cookies1,act1,flag1] = checked_logout(21,cookies1)        
        where1 = find_redirect(act1) 
        if flag1 != True:
            return
        
        [cookies2,act2,flag2] = checked_logout(21,cookies2)        
        where1 = find_redirect(act1) 
        if flag1 != True:
            return

        print("Test 21 Passed")
    except:
        print("Test 21 Exception Caused.")

In [24]:
# Test 22 - same user different cookies on second login
def test22():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(22,cookies1,"test1","password1")
        if flag1 != True:
            return
        
        [cookiesx,act1,flag1] = checked_logout(22,cookies1)        
        where1 = find_redirect(act1) 
        if flag1 != True:
            return
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(22,cookies2,"test1","password1")
        if flag2 != True:
            return
        
        [cookiesx,act2,flag2] = checked_logout(22,cookies2)        
        where1 = find_redirect(act1) 
        if flag2 != True:
            return
        
        if cookies1[1] == cookies2[1]:
            print("Test 22 Cookie magic reused for second login.")
            return      
        

        print("Test 22 Passed")
    except:
        print("Test 22 Exception Caused.")

In [25]:
# Test 23 - two users sessions should not overlap totals
def test23():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(23,cookies1,"test1","password1")
        if flag1!=True:
            return
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(23,cookies2,"test2","password2")
        if flag2 != True:
            return
        
        [cookies1,act] = do_add(cookies1,"road",1,"car")
        
        where = find_redirect(act)
    
        if where != None:
            print("Test 23 Unexpected redirect.")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 23 No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 23 No total message present")
            return
        
        if total != "1":
            print("Test 23 Total not expected value 1 was "+total)
            return

        [cookies2,act] = do_add(cookies2,"road",1,"bus")
            
        where = find_redirect(act)
    
        if where != None:
            print("Test 23 Unexpected redirect.")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 23 No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 23 No total message present")
            return
        
        if total != "1":
            print("Test 23 Total not expected value 1 was "+total)
            return
        
        [cookies2,act] = do_add(cookies2,"road",1,"bicycle")
            
        where = find_redirect(act)
    
        if where != None:
            print("Test 23 Unexpected redirect.")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 23 No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 23 No total message present")
            return
        
        if total != "2":
            print("Test 23 Total not expected value 2 was "+total)
            return

        [cookies1,act1,flag1] = checked_logout(23,cookies1)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return
        
        [cookies2,act2,flag2] = checked_logout(23,cookies2)        
        where1 = find_redirect(act1) 
        if flag1 != True:
            return

        print("Test 23 Passed")
    except:
        print("Test 23 Exception Caused.")

In [26]:
# Test 24 - Simple add with malformed occupancy
def test24():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(24,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return
        
        [cookies,act] = do_add(cookies,"road","1","car")
        initial_total = find_refill(act,'total')
        
        # here comes the bad request
        [cookies,act] = do_add(cookies,"road","1two","car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 24 Unexpected redirect.(2)")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 24 No message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 24 No total present")
            return
        
        if total != initial_total:
            print("Test 24 Total not expected value")
            return
        
        [cookies,act,flag] = checked_logout(24,cookies)        
        if flag != True:
            return
        
        print("Test 24 Passed")
    except:
        print("Test 24 Exception Caused.")

In [27]:
# Test 25 - Simple undo with malformed occupancy
def test25():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(25,cookies,"test1","password1")
        where = find_redirect(act)
        if flag != True:
            return
        
        [cookies,act] = do_add(cookies,"road","1","car")
        initial_total = find_refill(act,'total')
        
        # here comes the bad request
        [cookies,act] = do_undo(cookies,"road","1two","car")
      
        where = find_redirect(act)
    
        if where != None:
            print("Test 25 Unexpected redirect.(2)")
            return
        
        message = find_refill(act,'message')
    
        if message == None:
            print("Test 25 No message present")
            return
        
        total = find_refill(act,'total')
    
        if total == None:
            print("Test 25 No total present")
            return
        
        if total != initial_total:
            print("Test 25 Total not expected value")
            return
        
        [cookies,act,flag] = checked_logout(25,cookies)         
        if flag != True:
            return
        
        print("Test 25 Passed")
    except:
        print("Test 25 Exception Caused.")

In [28]:
# Run all tests
test1()
test2()
test3()
test4()
test5()
test6()
test7()
test8()
test9()
test10()
test11()
test12()
test13()
test14()
test15()
test16()
test17()
test18()
test19()
test20()
test21()
test22()
test23()
test24()
test25()

Test 1 Passed
Test 2 Passed
Test 3 Passed
Test 4 Passed
Test 5 Passed
Test 6 Passed
Test 7 Passed
Test 10 Passed
Test 13 Passed
Test 15 Passed
Test 16 Passed
Test 17 Passed
Test 18 Passed
Test 19 Passed
Test 20 Passed
Test 21 Passed
Test 22 Passed
Test 23 Passed
